In [1]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.functional as F
import torch.nn.functional as F

from scipy.spatial import distance

In [44]:
corpus = [
    'he is a king',
    'she is a queen',
    'he is a man',
    'she is a woman',
    'warsaw is poland capital',
    'berlin is germany capital',
    'paris is france capital',   
]

In [45]:
def tokenize_corpus(corpus):
    tokens = [x.split() for x in corpus]
    return tokens

tokenized_corpus = tokenize_corpus(corpus)

In [4]:
vocabulary = []
for sentence in tokenized_corpus:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)

In [5]:
window_size = 2
idx_pairs = []
for sentence in tokenized_corpus:
    indices = [word2idx[word] for word in sentence]
    for center_word_pos in range(len(indices)):
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make soure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

idx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array

In [6]:
def get_input_layer(word_idx):
    x = torch.zeros(vocabulary_size).float()
    x[word_idx] = 1.0
    return x

In [7]:
embedding_dims = 5
W1 = Variable(torch.randn(embedding_dims, vocabulary_size).float(), requires_grad=True)
W2 = Variable(torch.randn(vocabulary_size, embedding_dims).float(), requires_grad=True)
num_epochs = 1001
learning_rate = 0.001

for epo in range(num_epochs):
    loss_val = 0
    for data, target in idx_pairs:
        x = Variable(get_input_layer(data)).float()
        y_true = Variable(torch.from_numpy(np.array([target])).long())

        z1 = torch.matmul(W1, x)
        z2 = torch.matmul(W2, z1)    
        log_softmax = F.log_softmax(z2, dim=0)

        loss = F.nll_loss(log_softmax.view(1,-1), y_true)
        loss_val += loss.data.item()
        loss.backward()
    
        with torch.no_grad():
            W1 -= learning_rate * W1.grad
            W2 -= learning_rate * W2.grad

            W1.grad.zero_()
            W2.grad.zero_()

    if epo % 10 == 0:    
        print('Loss at epo {0}: {1}'.format(epo, loss_val/len(idx_pairs))) #'Loss at epo {epo}: {loss_val/len(idx_pairs)}'

Loss at epo 0: 6.8664751699992586
Loss at epo 10: 5.437300780841283
Loss at epo 20: 4.77494215965271
Loss at epo 30: 4.387421384879521
Loss at epo 40: 4.109059355940137
Loss at epo 50: 3.892965911115919
Loss at epo 60: 3.7176239899226595
Loss at epo 70: 3.571108218601772
Loss at epo 80: 3.4462477394512723
Loss at epo 90: 3.33836875643049
Loss at epo 100: 3.244168015888759
Loss at epo 110: 3.16115779876709
Loss at epo 120: 3.087382744039808
Loss at epo 130: 3.0212683745792934
Loss at epo 140: 2.9615271261760165
Loss at epo 150: 2.9070978283882143
Loss at epo 160: 2.8571009039878845
Loss at epo 170: 2.810803851059505
Loss at epo 180: 2.767595742430006
Loss at epo 190: 2.726966186932155
Loss at epo 200: 2.688487502506801
Loss at epo 210: 2.65180424451828
Loss at epo 220: 2.616621516432081
Loss at epo 230: 2.582699012756348
Loss at epo 240: 2.5498445408684867
Loss at epo 250: 2.5179105503218513
Loss at epo 260: 2.4867896352495467
Loss at epo 270: 2.456411632469722
Loss at epo 280: 2.426739

In [8]:
# W1.shape, W2.shape

In [9]:
with torch.no_grad():
    emb = {word: torch.matmul(W1, get_input_layer(i)).numpy() for i, word in enumerate(vocabulary)}

In [10]:
print(distance.cosine(emb['man'], emb['woman']))
print(distance.cosine(emb['man'], emb['he']))
print(distance.cosine(emb['man'], emb['she']))
print(distance.cosine(emb['woman'], emb['she']))
print()
print(distance.cosine(emb['king'], emb['he']))
print(distance.cosine(emb['king'], emb['she']))
print(distance.cosine(emb['king'], emb['queen']))
print(distance.cosine(emb['queen'], emb['he']))
print(distance.cosine(emb['queen'], emb['she']))
print()
print(distance.cosine(emb['warsaw'], emb['berlin']))
print(distance.cosine(emb['warsaw'], emb['paris']))
print(distance.cosine(emb['warsaw'], emb['poland']))
print(distance.cosine(emb['warsaw'], emb['germany']))
print(distance.cosine(emb['berlin'], emb['germany']))
print(distance.cosine(emb['berlin'], emb['poland']))
print(distance.cosine(emb['berlin'], emb['france']))

0.2198285460472107
0.2600386142730713
0.06145864725112915
0.3356438875198364

0.03295016288757324
0.17506688833236694
0.6612101793289185
0.5454409420490265
0.2948251962661743

0.17275750637054443
0.9971998913679272
0.7850916236639023
0.7926154434680939
0.8764673694968224
1.3115880191326141
0.9882218223065138


In [11]:
with torch.no_grad():
    emb = {word: W1[:, i] * W2[i] for i, word in enumerate(vocabulary)}

In [12]:
print(distance.cosine(emb['man'], emb['woman']))
print(distance.cosine(emb['man'], emb['he']))
print(distance.cosine(emb['man'], emb['she']))
print(distance.cosine(emb['woman'], emb['she']))
print()
print(distance.cosine(emb['king'], emb['he']))
print(distance.cosine(emb['king'], emb['she']))
print(distance.cosine(emb['king'], emb['queen']))
print(distance.cosine(emb['queen'], emb['he']))
print(distance.cosine(emb['queen'], emb['she']))
print()
print(distance.cosine(emb['warsaw'], emb['berlin']))
print(distance.cosine(emb['warsaw'], emb['paris']))
print(distance.cosine(emb['warsaw'], emb['poland']))
print(distance.cosine(emb['warsaw'], emb['germany']))
print(distance.cosine(emb['berlin'], emb['germany']))
print(distance.cosine(emb['berlin'], emb['poland']))
print(distance.cosine(emb['berlin'], emb['france']))

0.052087247371673584
0.5862727761268616
0.17826193571090698
0.08458787202835083

0.03320729732513428
0.3027147650718689
0.47494280338287354
0.6454004049301147
0.3522189259529114

1.5133419036865234
1.2108029425144196
0.9838755503296852
0.8900401517748833
0.6909601092338562
1.3766942024230957
1.4267953634262085


### same things in real dataset

In [77]:
import gzip
import gensim

import re

from collections import Counter
from tqdm import tqdm

from collections import OrderedDict

import pickle

In [17]:
def read_input(input_file):
    """This method reads the input file which is in gzip format"""

    with gzip.open(input_file, 'rb') as f:
        for i, line in enumerate(f):

            # do some pre-processing and return list of words for each review
            # text
            yield gensim.utils.simple_preprocess(line)

In [20]:
data_file = 'opinrankdatasetwithjudgments.tar.gz' # 'OpinRankDatasetWithJudgments.zip'
documents = list(read_input (data_file))

In [34]:
documents[36]

['query', 'good', 'cleanliness', 'nice', 'room', 'excellent', 'staff']

In [36]:
c = Counter()
for line in tqdm(documents):
    for word in line:
        c[word] += 1

100%|██████████| 792362/792362 [00:21<00:00, 37601.16it/s] 


In [58]:
c.most_common(10)

[('the', 3095891),
 ('and', 1631827),
 ('to', 1175956),
 ('was', 927119),
 ('in', 805277),
 ('we', 677762),
 ('of', 671223),
 ('is', 627155),
 ('for', 612576),
 ('it', 600419)]

In [71]:
# vocabulary = []
# for sentence in tqdm(documents):
#     for token in sentence:
#         if token not in vocabulary:
#             vocabulary.append(token)

# word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
# idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

# vocabulary_size = len(vocabulary)

In [60]:
def ha(source):
    pattern = r'[ \'\(\)\[\]]'
    return '{}'.format(re.sub(pattern, '', str(source)))

In [61]:
ha(tokenized_corpus).split(',')

['he',
 'is',
 'a',
 'king',
 'she',
 'is',
 'a',
 'queen',
 'he',
 'is',
 'a',
 'man',
 'she',
 'is',
 'a',
 'woman',
 'warsaw',
 'is',
 'poland',
 'capital',
 'berlin',
 'is',
 'germany',
 'capital',
 'paris',
 'is',
 'france',
 'capital']

In [62]:
list(OrderedDict.fromkeys(ha(tokenized_corpus).split(',')).keys())

['he',
 'is',
 'a',
 'king',
 'she',
 'queen',
 'man',
 'woman',
 'warsaw',
 'poland',
 'capital',
 'berlin',
 'germany',
 'paris',
 'france']

In [64]:
all_words = ha(documents).split(',')

In [65]:
all_words[:3]

['ustar', 'webuser', 'webuser']

In [68]:
vocabulary = list(OrderedDict.fromkeys(all_words).keys())

In [73]:
vocabulary_size = len(vocabulary)
word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

In [74]:
vocabulary_size

189478

In [75]:
window_size = 4
idx_pairs = []
for sentence in tqdm(documents):
    indices = [word2idx[word] for word in sentence]
    for center_word_pos in range(len(indices)):
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make soure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

idx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array


 25%|██▌       | 200318/792362 [00:42<02:04, 4748.06it/s]


 29%|██▊       | 227445/792362 [01:12<02:58, 3157.69it/s]


 32%|███▏      | 251200/792362 [01:42<03:41, 2448.53it/s]


 36%|███▌      | 282250/792362 [02:13<04:01, 2109.99it/s]


 39%|███▉      | 309396/792362 [05:17<08:15, 975.34it/s]


 42%|████▏     | 336501/792362 [05:51<07:56, 957.54it/s]


 46%|████▋     | 366691/792362 [06:22<07:23, 959.36it/s]


 50%|█████     | 397699/792362 [06:52<06:49, 963.37it/s]


 72%|███████▏  | 572184/792362 [07:25<02:51, 1285.04it/s]


100%|██████████| 792362/792362 [07:54<00:00, 1668.35it/s]

In [78]:
with open('idx_pairs.pickle', 'wb') as f:
    pickle.dump(idx_pairs, f)

OverflowError: cannot serialize a bytes object larger than 4 GiB

In [84]:
embedding_dims = 128
W1 = Variable(torch.randn(embedding_dims, vocabulary_size).float(), requires_grad=True)
W2 = Variable(torch.randn(vocabulary_size, embedding_dims).float(), requires_grad=True)
num_epochs = 101
learning_rate = 0.001

for epo in range(num_epochs):
    loss_val = 0
    for data, target in idx_pairs:
        x = Variable(get_input_layer(data)).float()
        y_true = Variable(torch.from_numpy(np.array([target])).long())

        z1 = torch.matmul(W1, x)
        z2 = torch.matmul(W2, z1)    
        log_softmax = F.log_softmax(z2, dim=0)

        loss = F.nll_loss(log_softmax.view(1,-1), y_true)
        loss_val += loss.data.item()
        loss.backward()
    
        with torch.no_grad():
            W1 -= learning_rate * W1.grad
            W2 -= learning_rate * W2.grad

            W1.grad.zero_()
            W2.grad.zero_()

#     if epo % 10 == 0:    
#         print('Loss at epo {0}: {1}'.format(epo, loss_val/len(idx_pairs))) #'Loss at epo {epo}: {loss_val/len(idx_pairs)}'
    print('Loss at epo {0}: {1}'.format(epo, loss_val/len(idx_pairs)))

KeyboardInterrupt: 

In [5]:
import time

In [9]:
current_time = time.time()
time.sleep(5)
time.time() - current_time

5.003559827804565